## 問題識別器の準備

In [1]:
INPUT_MAX_LEN=512
TEST_DATASET = "./data/test.csv"
PRETRAINED_MODEL_NAME = "cl-tohoku/bert-base-japanese-char-whole-word-masking"
DIS_CHECKPOINT = "./checkpoint/best_loss_epoch=2.ckpt"

In [2]:
import pandas as pd
from transformers import AutoTokenizer
import torch
from torch import optim
from torch import nn
import pytorch_lightning as pl

In [3]:
class FineTuneBert(pl.LightningModule):
    def __init__(self, model_name_or_path, n_classes, n_epochs=None):
        super().__init__()

        self.model = BertModel.from_pretrained(model_name_or_path, return_dict=True)
        self.classifier = nn.Linear(self.model.config.hidden_size, n_classes)
        self.n_epoch = n_epochs
        self.criterion = nn.CrossEntropyLoss()

        ## BertLayerモジュールの最後を勾配計算ありに変更
        for param in self.model.parameters():
            param.requires_grad = False
        for param in self.model.encoder.layer[-1].parameters():
            param.requires_grad = True

    def forward(self, input_ids, attention_mask, labels=None):
        output = self.model(input_ids, attention_mask=attention_mask)
        preds = self.classifier(output.pooler_output)
        loss = 0
        if labels is not None:
            loss = self.criterion(preds, labels)

        return loss, preds

In [4]:
import torch
from transformers import BertModel, AutoTokenizer
from torchvision import models

# トークナイザー
tokenizer = AutoTokenizer.from_pretrained(PRETRAINED_MODEL_NAME)

# 学習済モデル
trained_model = FineTuneBert(PRETRAINED_MODEL_NAME, n_classes=2)
checkpoint = torch.load(DIS_CHECKPOINT, map_location=torch.device('cpu'))
trained_model.load_state_dict(checkpoint["state_dict"])

# GPU利用の有無
USE_GPU = torch.cuda.is_available()
if USE_GPU:
    trained_model.cuda()

/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


## データセットの準備

In [5]:
import pandas as pd

# データセットの読み込み
df = pd.read_csv(TEST_DATASET)

## 問題識別

In [6]:
trained_model.eval()

for index, row in df.iterrows():
    question = row["question"]
    answer = row["answer"]
    label = row["tf"]

    input = question + "答えは" + answer

    ## tokenize
    tokenized_input = tokenizer.batch_encode_plus(
        [input],
        max_length=INPUT_MAX_LEN,
        truncation=True,
        add_special_tokens=True,
        padding="longest",
        return_tensors="pt"
    )

    input_ids = tokenized_input["input_ids"]
    input_mask = tokenized_input["attention_mask"]

    ## デバイスの指定
    if torch.cuda.is_available():
        input_ids = input_ids.cuda()
        input_mask = input_mask.cuda()

    ## 識別
    with torch.no_grad():
        _, score = trained_model(input_ids, input_mask)
        pred = torch.argmax(score)
        if pred == 1:
            print(f"pred: {pred}, ans: {label}")
            print(question)

pred: 1, ans: 1
次の文章は正しいか答えなさい。ネットワークを利用したサイバー犯罪には次のような特徴がある。1.匿名性が高い2.痕跡が残りやすい3.不特定多数が被害を受ける4.時間と空間の制約がある
pred: 1, ans: 1
以下の文章は正しいか答えてください。数式に表すことも一種のモデル化である。
pred: 1, ans: 1
統合技法とはどのような方法ですか?
pred: 1, ans: 1
次の文章は正しいか答えなさい。特許権、実用新案権、意匠権を総称して産業財産権という。
pred: 1, ans: 1
次の文章は正しいか答えよ。クレジットカードでは使用者は利用前に支払いを行う。
pred: 1, ans: 1
産業財産権にはどのようなものがありますか
pred: 1, ans: 1
次の文章は正しいか答えなさい。一般利用者が情報システムを利用するにあたり、設定するパスワードは通常、英数字、記号などを組み合わせ、他の人に容易に推測されない複雑なものにする必要がある。
pred: 1, ans: 1
キャッシュレス化が進む中国や韓国ではどのような決済方法が利用されていますか
pred: 1, ans: 1
次の文章は正しいか答えなさい。態度技法は集めたデータをまとめていくことで有効な情報を形成していき、ロールプレイング、ワールドカフェがその代表である。
pred: 1, ans: 1
サミュエル・d・ウォーレンスとルイス・d・ブランダイスはどのような権利を主張したか?
pred: 1, ans: 0
コンピュータでは何ビットで表現できますか?
pred: 1, ans: 1
次の文章は正しいですか?ベクタ系画像処理ソフトでは、円や長方形などの図形はオブジェクトという一つの塊として描画される。
pred: 1, ans: 1
次の文章は正しいか答えなさい。教育を担任する者、授業を受ける者は、授業の過程で利用するために著作物を複製することが違法になる。
pred: 1, ans: 1
サミュエル・d・ウォーレンスとルイス・d・ブランダイスは19世紀末に何を主張したか
pred: 1, ans: 1
次の文章は正しいか答えなさい。ssdのかわりにハードディスクを使うとコンピュータの実質的な動作速度を上げることができる。
pred: 1, ans